# <b>背景</b>

<p>ET（essential tremor）患者：原发性震颤(Essential tremor)患者，特发性震颤（essential tremor，ＥＴ）最常见的运动障碍性疾病，主要为手、头部及身体其他部位的姿位性和运动性震颤。发病部位：上肢、头、面部、下颚。</p>
<p>参见文献： louis2003 Factors associated with increased risk of head tremor in essential tremor_ a community-based study in northern Manhattan）。</p>
<p>中线震颤（midline tremor）：   包含：面部（下颌部+唇部）、舌头、声音、头部（又称颈部）和躯干。<p>

    
# <b>目的</b>
1.1.	探索ET患者伴中线震颤的危险因素。
    
1.2.	ET患者伴焦虑和抑郁的危险因素    
    
# <b>任务</b>

该notebook主要分析变量的重要性和相关性。以便组合变量（避免变量之间的共线性），用于逻辑回归模型评估变量的影响。


# <b>第一步 引包</b>

In [17]:
#引包：引入所需python包
import xlrd
import os
import re
import pandas as pd
import numpy as np
import itertools
from scipy import stats
from scipy.stats import kstest
from scipy.stats import chi2_contingency
from scipy.stats import chisquare
from scipy.stats import mannwhitneyu

import matplotlib as mpl
from matplotlib import pyplot as plt
from numpy import nan

import seaborn as sns
import time

from imblearn.over_sampling import SMOTE
#随机森林
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)

from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve,auc
from sklearn import linear_model, datasets
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import collections  #count frequence of items in list

import warnings
warnings.filterwarnings('ignore')

# <b>第二步 读取清洗后数据</b>

1）设置默认目录

2）读取csv文件；


In [18]:
dir = "./"
print(os.listdir(dir))

['.ipynb_checkpoints', '1.数据读取和清洗.ipynb', 'data', 'output', '2.中线震颤并发情况.ipynb', '3.人口特征统计.ipynb', '4.疾病特征统计.ipynb', '5.变量相关性和重要性.ipynb', '6.危险因子（逻辑回归）.ipynb', 'requirements.txt', 'pip_search_result.txt']


In [19]:
#清洗后数据
df = pd.read_csv(dir+"output/df_clean.csv",index_col=0) #第一列为行索引
#df = df.drop(columns = df.columns[0]) #删除不需要的列
df.head(2)

,编号,性别,工作状态合并栏,婚姻状况合并栏,年龄,发病年龄,主要受累部位=2,总病程,上肢病程,下肢病程,...,声音,颈部,面声颈部位分级分数,面声颈量表分数,面声颈有无,声颈量表部位分级,声颈量表分数,声颈有无,下肢震颤,意向性震颤
0,G001,0,1,1,72,68,0,4.0,4.000,0.001,...,0,0,0,0,0,0,0,0,1,0
1,G002,0,1,1,45,40,1,5.0,0.001,0.001,...,1,1,2,3,1,2,3,1,1,0


In [20]:
#中线震颤数据
midline1 = pd.read_csv(dir+"output/midline1.csv",index_col=0) #第一列为行索引
#midline1 = midline1.drop(columns = midline1.columns[0]) #删除不需要的列
midline1.head(2)

,编号,静止性上肢震颤分数,运动性上肢震颤总分,运动性下肢震颤分数,运动性四肢震颤总分,上肢静止性震颤,下肢静止性震颤,四肢静止性震颤,意向性震颤,面部,...,下肢震颤,抑郁分类,焦虑分类,运动性上肢震颤总分有无,运动性下肢震颤分数有无,运动性四肢震颤总分有无,面声颈部位分级分数有无,面声颈量表分数有无,声颈量表部位分级有无,声颈量表分数有无
0,G001,0,21,6,27,0,0,0,0,0,...,1,0,0,1,1,1,0,0,0,0
1,G002,0,13,2,15,0,0,0,0,0,...,1,0,0,1,1,1,1,1,1,1


# <b>第三步 准备数据 </b>
根据数据分析任务，处理数据

<b> 1) 筛选数据 </b>

<b>筛选与疾病有关字段</b>

In [21]:
dem = df.filter(items=["编号","性别",'工作状态合并栏', '婚姻状况合并栏','受教育时间',"婚姻状况","年龄","发病年龄","总病程","上肢病程","下肢病程","颈部病程","声音病程","面部病程","舌病程","躯干病程"])
dem = pd.merge(dem,midline1,on=["编号"] ,how = "left")
dem.to_csv("output/dem.csv")
dem.head(2)

,编号,性别,工作状态合并栏,婚姻状况合并栏,受教育时间,年龄,发病年龄,总病程,上肢病程,下肢病程,...,下肢震颤,抑郁分类,焦虑分类,运动性上肢震颤总分有无,运动性下肢震颤分数有无,运动性四肢震颤总分有无,面声颈部位分级分数有无,面声颈量表分数有无,声颈量表部位分级有无,声颈量表分数有无
0,G001,0,1,1,6,72,68,4.0,4.000,0.001,...,1,0,0,1,1,1,0,0,0,0
1,G002,0,1,1,16,45,40,5.0,0.001,0.001,...,1,0,0,1,1,1,1,1,1,1


<b>筛选与人口有关字段</b>

In [22]:
inf = df.filter(items=["编号",'主观认知功能下降', 'MMSE',
       '家族史', '高血压', '糖尿病', '其他', '抗ET药物使用', '抗焦虑抑郁药物使用', '吸烟', '饮酒', 
        'TRS C:15-22',  '自述焦虑时长','HAMA总', '自述抑郁时长', 'HAMD总分', '抑郁分类', '3级', '匹兹堡总分','焦虑分类', 'HAMA3级'])  #,'面声颈部位分级分数','声颈量表部位分级'
inf.rename(columns = {"TRS C:15-22":"TRS_C","3级":"HAMD分级3级",'抑郁分类':'HAMD分级2级',"焦虑分类":"HAMA分级2级"},inplace = True)
inf = pd.merge(inf,midline1,on=["编号"] ,how = "left")

inf.head(2)

,编号,主观认知功能下降,MMSE,家族史,高血压,糖尿病,其他,抗ET药物使用,抗焦虑抑郁药物使用,吸烟,...,下肢震颤,抑郁分类,焦虑分类,运动性上肢震颤总分有无,运动性下肢震颤分数有无,运动性四肢震颤总分有无,面声颈部位分级分数有无,面声颈量表分数有无,声颈量表部位分级有无,声颈量表分数有无
0,G001,记忆力稍下降,27,0,0,0,0,0,0,0,...,1,0,0,1,1,1,0,0,0,0
1,G002,轻微下降,29,0,0,0,0,0,0,0,...,1,0,0,1,1,1,1,1,1,1


In [23]:
#清洗病程
for i in ["主观认知功能下降","高血压","糖尿病","其他","抗ET药物使用","抗焦虑抑郁药物使用","吸烟","饮酒","自述焦虑时长","自述抑郁时长"]:  #,"HAMD分级3级",'HAMD分级2级', "HAMA分级2级"]:
    print (i)
    inf[i] = np.where(inf[i].str.contains(r"\D"),1,0)
    inf[i] = inf[i].astype(float)
inf.to_csv("output/inf.csv")
inf.head(10)

主观认知功能下降
高血压
糖尿病
其他
抗ET药物使用
抗焦虑抑郁药物使用
吸烟
饮酒
自述焦虑时长
自述抑郁时长


,编号,主观认知功能下降,MMSE,家族史,高血压,糖尿病,其他,抗ET药物使用,抗焦虑抑郁药物使用,吸烟,...,下肢震颤,抑郁分类,焦虑分类,运动性上肢震颤总分有无,运动性下肢震颤分数有无,运动性四肢震颤总分有无,面声颈部位分级分数有无,面声颈量表分数有无,声颈量表部位分级有无,声颈量表分数有无
0,G001,1.0,27,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,1,1,0,0,0,0
1,G002,1.0,29,0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,1,1,1,1,1,1
2,G003,1.0,27,0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,0,0,1,1,1,0,0,0,0
3,G004,0.0,29,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,1,0,1,1,1,1,1
4,G005,1.0,27,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,1,0,1,0,0,0,0
5,G006,1.0,28,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,1,1,1,1,1,1,1
6,G007,1.0,23,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,1,0,1,1,1,1,1
7,G008,0.0,29,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,1,0,1,1,1,1,1
8,G009,1.0,26,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,1,1,1,1,1,1
9,G010,1.0,27,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,1,0,1,1,1,1,1


In [24]:
data_0 = pd.merge(dem,inf,how = "left")                         #.merge(df.filter(items = ['编号', '姓名', '面声颈量表分数', '声颈量表分数']))
data_0.columns

Index(['编号', '性别', '工作状态合并栏', '婚姻状况合并栏', '受教育时间', '年龄', '发病年龄', '总病程', '上肢病程',
       '下肢病程', '颈部病程', '声音病程', '面部病程', '舌病程', '躯干病程', '静止性上肢震颤分数', '运动性上肢震颤总分',
       '运动性下肢震颤分数', '运动性四肢震颤总分', '上肢静止性震颤', '下肢静止性震颤', '四肢静止性震颤', '意向性震颤',
       '面部', '声音', '颈部', '面声颈部位分级分数', '面声颈量表分数', '面声颈有无', '声颈量表部位分级', '声颈量表分数',
       '声颈有无', '下肢震颤', '抑郁分类', '焦虑分类', '运动性上肢震颤总分有无', '运动性下肢震颤分数有无',
       '运动性四肢震颤总分有无', '面声颈部位分级分数有无', '面声颈量表分数有无', '声颈量表部位分级有无', '声颈量表分数有无',
       '主观认知功能下降', 'MMSE', '家族史', '高血压', '糖尿病', '其他', '抗ET药物使用', '抗焦虑抑郁药物使用',
       '吸烟', '饮酒', 'TRS_C', '自述焦虑时长', 'HAMA总', '自述抑郁时长', 'HAMD总分', 'HAMD分级2级',
       'HAMD分级3级', '匹兹堡总分', 'HAMA分级2级', 'HAMA3级'],
      dtype='object')

In [25]:
model_data = data_0.filter(items = ['编号', '姓名', '性别', '受教育时间','工作状态合并栏', '婚姻状况合并栏', '年龄', '发病年龄','HAMA总','HAMD总分','匹兹堡总分', 'TRS_C',
  'MMSE','家族史','抗焦虑抑郁药物使用','抗ET药物使用', '高血压', '糖尿病', '其他','吸烟', '饮酒','自述焦虑时长','自述抑郁时长',
  '主观认知功能下降','面声颈量表分数', '声颈量表分数','总病程','四肢静止性震颤','HAMD分级2级', 'HAMD分级3级',  'HAMA分级2级', 'HAMA3级'])
model_data.columns
#type = ["声音有无","颈部有无","面部有无","面声颈有无","四肢静止性震颤","声颈有无","抑郁分类","焦虑分类"]
model_data[model_data.isnull().values==True] #查看是否有空行

,编号,性别,受教育时间,工作状态合并栏,婚姻状况合并栏,年龄,发病年龄,HAMA总,HAMD总分,匹兹堡总分,...,自述抑郁时长,主观认知功能下降,面声颈量表分数,声颈量表分数,总病程,四肢静止性震颤,HAMD分级2级,HAMD分级3级,HAMA分级2级,HAMA3级


In [26]:
model_data.to_csv("output/model_data.csv")


# <b> 第四步 变量重要性（随机森林） </b>


In [27]:
list3 = ['上肢静止性震颤', '四肢静止性震颤','运动性下肢震颤分数有无', '下肢震颤','面部','声音','颈部','声颈有无',
'面声颈有无','抑郁分类','焦虑分类','意向性震颤']


In [28]:
for type in list3:
    print ("-------------------",type,"-------------------")
    if type not in ['抑郁分类', '焦虑分类']:
        X = model_data.drop(columns = ['编号','面声颈量表分数', '声颈量表分数']) 
        if type == "四肢静止性震颤":
            X = X.drop(columns = ["四肢静止性震颤"])
    else :
        X = model_data.drop(columns = ['编号', 'HAMA总','HAMD总分','HAMD分级2级', 'HAMD分级3级',  'HAMA分级2级', 'HAMA3级'])  
    y = data_0[type].astype(int)
    #SMOT

    os = SMOTE(random_state=0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    columns = X_train.columns

    os_data_X,os_data_y=os.fit_sample(X_train, y_train)
    os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
    os_data_y= pd.DataFrame(data=os_data_y,columns=['type'])
    data = os_data_X
    data["type"] = os_data_y
    # we can Check the numbers of our data
    print("length of oversampled data is ",len(os_data_X))
    print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['type']==0]))
    print("Number of subscription",len(os_data_y[os_data_y['type']==1]))
    #print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['type']==0])/len(os_data_X))
    #print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['type']==1])/len(os_data_X))
    collections.Counter(data["type"])
    model_data.head(2)
        
    feature_dataframe = pd.DataFrame()
    accuracy_RF = []
    for i in range(0,10):
        #print ("-- " ,i ,"time--")
        #sample_data = all_data.drop(all_data[all_data["type"]==1].sample(frac=0.5).index)   ,有 class weight
        #collections.Counter(all_1["type"])
        train,test = train_test_split(data, test_size = 1/3, random_state = 42)
        ntrain = train.shape[0]
        ntest = test.shape[0]     #抽样样本中随机再划分
        y_train = train['type'].ravel()
        train = train.drop(['type'], axis=1)  #dataframe
        x_train = train.values   # Creates an array of the train data
        x_test = test.drop(['type'], axis=1).values       # Creats an array of the test data
        y_test = test['type'].ravel()
    
        rf = RandomForestClassifier(class_weight='balanced', max_depth = 4, max_features = 'sqrt', min_samples_leaf = 2, n_estimators = 500).fit(x_train, y_train)
        predictions = rf.predict(x_test)
        accuracy1 = accuracy_score(y_test, predictions)
        #print("Accuracy: %.2f%%" % (accuracy * 100.0))
        importances = pd.DataFrame(rf.feature_importances_,train.columns)   #.T
        importances.columns = ["features importances"+str(i)]
        importances = importances.sort_values(by = "features importances"+str(i), ascending =False)

        accuracy1 = float(accuracy1)    
        accuracy_RF.append(accuracy1)    
        feature_dataframe = pd.concat([feature_dataframe,importances],axis = 1 )
        
    print ("Random forest mean accuracy: %.2f%%" % (np.mean(accuracy_RF)*100.0))    
    feature_dataframe ["mean_importance"] =feature_dataframe .mean(1)
    feature_dataframe  = feature_dataframe .sort_values(by = "mean_importance" , ascending =False)

    print(feature_dataframe["mean_importance"])
    #np.mean(accuracy_RF)

------------------- 上肢静止性震颤 -------------------
length of oversampled data is  256
Number of no subscription in oversampled data 128
Number of subscription 128
Random forest mean accuracy: 99.65%
四肢静止性震颤      0.380544
主观认知功能下降     0.137203
年龄           0.062436
TRS_C        0.060453
发病年龄         0.047450
抗ET药物使用      0.038410
HAMD分级3级     0.032052
工作状态合并栏      0.031045
总病程          0.028948
匹兹堡总分        0.028102
自述抑郁时长       0.020554
MMSE         0.017161
HAMD总分       0.016267
受教育时间        0.015487
饮酒           0.012066
家族史          0.011772
HAMD分级2级     0.007842
HAMA总        0.007564
HAMA3级       0.007118
HAMA分级2级     0.006769
自述焦虑时长       0.005738
抗焦虑抑郁药物使用    0.005679
吸烟           0.005441
性别           0.005250
糖尿病          0.004632
高血压          0.001552
婚姻状况合并栏      0.001261
其他           0.001206
Name: mean_importance, dtype: float64
------------------- 四肢静止性震颤 -------------------
length of oversampled data is  256
Number of no subscription in oversampled data 128
Number of subscri

# <b>第五步 相关性分析 </b>

In [30]:
data = model_data
data = data.fillna(0)
classfield =  ['性别',"工作状态合并栏",  '婚姻状况合并栏','家族史','抗焦虑抑郁药物使用','抗ET药物使用', '高血压', '糖尿病', '其他','吸烟', '饮酒','自述焦虑时长','自述抑郁时长',
  '主观认知功能下降', 'HAMD分级2级', 'HAMD分级3级', 'HAMA分级2级', 'HAMA3级']  #未做工作状态
continufield = ['年龄', '受教育时间','发病年龄','总病程','HAMA总','HAMD总分','匹兹堡总分','TRS_C', 'MMSE','面声颈量表分数', '声颈量表分数']

#连续变量
res_corr = pd.DataFrame()
for i in itertools.combinations(continufield,2):  
    x1 = data[i[0]] 
    y1 = data[i[1]]
    x = x1[x1.notnull() & y1.notnull()  ] 
    y = y1[x1.notnull() & y1.notnull() ] 
    n1 = kstest(x, 'norm')[1] 
    n2 = kstest(y, 'norm')[1]
    p_lev = stats.levene(x,y)[1]  #方差齐性  ,检验  
    p_pearson=stats.pearsonr(x,y)[1]
    p_spear =stats.spearmanr(x,y)[1]    
    row1 =[[i[0],i[1],n1,n2,p_lev,p_pearson,p_spear]]
    res_corr = res_corr.append(row1)
res_corr.columns = ["字段1","字段2","字段1正态性检验，p值","字段2正态性检验，p值","方差齐性检验，p值","pearson_p值","spearman_p值"]
res_corr["pearson相关性显著"] = (res_corr["pearson_p值"]>0.05)   #相关性只能用于连续变量
res_corr["spearman相关性显著"] = (res_corr["spearman_p值"]>0.05)
res_corr.head(2)

#res_corr.to_csv("../internal/field_test_continu.csv")
res_corr.head(4)

,字段1,字段2,字段1正态性检验，p值,字段2正态性检验，p值,方差齐性检验，p值,pearson_p值,spearman_p值,pearson相关性显著,spearman相关性显著
0,年龄,受教育时间,0.0,2.564185e-258,5.940441e-34,9.398438e-10,5.848225e-10,False,False
0,年龄,发病年龄,0.0,0.000000e+00,3.004315e-01,1.099439e-58,6.934225e-57,False,False
0,年龄,总病程,0.0,0.000000e+00,6.869465e-15,2.124035e-02,2.733056e-01,False,True
0,年龄,HAMA总,0.0,4.943339e-198,3.698073e-23,1.393372e-01,2.459999e-01,True,True


In [31]:
#分类变量
import itertools
result  =pd.DataFrame()  
from scipy.stats import chi2_contingency
from scipy.stats import chisquare
for i in itertools.combinations(classfield,2):
    data1 = pd.crosstab(data[i[0]],data[i[1]],margins = True) 
    g, p, t,k, = chi2_contingency(data1)   #chi2_contingency(data)   #第一个值为卡方值，第二个值为P值，第三个值为自由度，第四个为与原数据数组同维度的对应理论值
    line = [(i[0],i[1],g,p)]   #chisquare(data,axis =None )频数
    #print (line)
    result = result.append(line) #未做“工作状态”的检验，因频数太小，不满足
#result.to_csv("../internal/field_test_class.csv")
result.columns = ["字段1","字段2","统计值","p值"]
result.head(4)

,字段1,字段2,统计值,p值
0,性别,工作状态合并栏,0.070972,0.999385
0,性别,婚姻状况合并栏,1.640894,0.801424
0,性别,家族史,0.356506,0.985880
0,性别,抗焦虑抑郁药物使用,2.198898,0.699231


In [32]:
##  连续变量和类别变量之间关系 One-way ANOVA test   H0  ： two or more groups have the same population mean
result  =pd.DataFrame() 
for i in  classfield:
    for j in continufield:        
        data1 = data.loc[data[i] ==0,j].values
        data2 = data.loc[data[i] ==1,j].values
        s,p = stats.f_oneway(data1,data2) 
        line = [(i,j,s,p)]
        result = result.append(line) #未做“工作状态”的检验，因频数太小，不满足
result.columns = ["字段1","字段2","统计值","p值"]
result.head(4)
#result.to_csv("../internal/field_test_continuclass.csv")

,字段1,字段2,统计值,p值
0,性别,年龄,3.302177,0.070700
0,性别,受教育时间,11.425792,0.000872
0,性别,发病年龄,2.234258,0.136573
0,性别,总病程,0.217675,0.641330


In [33]:
##  连续变量和类别变量之间关系 One-way ANOVA test   H0  ： two or more groups have the same population mean
result  =pd.DataFrame() 
for i in  ['工作状态合并栏', '婚姻状况合并栏']:
    for j in continufield:        
        data1 = data.loc[data[i] ==1,j].values
        data2 = data.loc[data[i] ==2,j].values
        s,p = stats.f_oneway(data1,data2) 
        line = [(i,j,s,p)]
        result = result.append(line) #未做“工作状态”的检验，因频数太小，不满足
result.columns = ["字段1","字段2","统计值","p值"]
result

,字段1,字段2,统计值,p值
0,工作状态合并栏,年龄,48.144602,5.506514e-11
0,工作状态合并栏,受教育时间,26.014388,7.905449e-07
0,工作状态合并栏,发病年龄,35.408928,1.195053e-08
0,工作状态合并栏,总病程,0.510812,4.756294e-01
0,工作状态合并栏,HAMA总,0.458571,4.990829e-01
0,工作状态合并栏,HAMD总分,0.737718,3.914316e-01
0,工作状态合并栏,匹兹堡总分,1.379690,2.415647e-01
0,工作状态合并栏,TRS_C,3.168350,7.661193e-02
0,工作状态合并栏,MMSE,10.340807,1.519754e-03
0,工作状态合并栏,面声颈量表分数,0.549477,4.594098e-01
